In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
from selenium.webdriver.common.by import By

import pandas as pd
import time

In [11]:
coordinates = ['ne=24.820003%2C46.644559&q=Food&sw=24.75486%2C46.539502', 'ne=24.84493%2C46.737685&q=Food&sw=24.779799%2C46.632628', 'ne=24.847033%2C46.825576&q=Food&sw=24.781903%2C46.720519',
               'ne=24.786189%2C46.86832&q=Food&sw=24.721028%2C46.763263', 'ne=24.713933%2C46.874671&q=Food&sw=24.648733%2C46.769614', 'ne=24.673694%2C46.876817&q=Food&sw=24.608474%2C46.77176', 
               'ne=24.670886%2C46.787295&q=Food&sw=24.605664%2C46.682239', 'ne=24.65942%2C46.70507&q=Food&sw=24.594193%2C46.600013', 'ne=24.731085%2C46.701379&q=Food&sw=24.665894%2C46.596322',
               'ne=24.792968%2C46.712623&q=Food&sw=24.72781%2C46.607566', 'ne=24.81603%2C46.781631&q=Food&sw=24.750885%2C46.676574', 'ne=24.71097%2C46.782146&q=Food&sw=24.645769%2C46.677089',
               'ne=24.902942%2C46.673055&q=Food&sw=24.837842%2C46.567998','ne=24.923493%2C46.90115&price=4&q=Food&sw=24.66293%2C46.480923', 'ne=24.79889%2C47.106113&q=Food&sw=24.532445%2C46.685886']

In [12]:

driver = webdriver.Chrome()
columns_ = ['Name', 'Score', 'Category', 'Price_Range']

df = pd.DataFrame(columns=columns_)

for n in coordinates:
    url = f'https://foursquare.com/explore?mode=url&{n}'
    driver.get(url)
    time.sleep(2)
    for i in range(2, 12):  # Adjust the range to try 10 pages
        # Request the page
        
        # Extract the body and parse it
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract names, scores, categories, and price ranges from the page
        names = soup.find_all('div', {'class': 'venueName'})
        scores = soup.find_all('div', {'class': 'venueScore positive', 'title': '', 'style': 'background-color: #00B551;'})
        categories = soup.find_all('span', {'class': 'categoryName'})
        price_ranges = soup.find_all('span', {'class': 'venueDataItem'})

        # Clean the data
        clean_Name = [p.text.strip() for p in names]
        clean_Score = [n.text.strip() for n in scores]
        clean_Category = [m.text.strip() for m in categories]

        # Extract and clean price ranges
        clean_Price_Range = []
        for span in price_ranges:
            price_span = span.find('span', {'class': 'price'})
            if price_span:
                price_text = price_span.find('span', {'class': 'darken'})
                if price_text:
                    clean_Price_Range.append(price_text.text.strip())


        # Check if all lists have the same length before creating the DataFrame
        min_length = min(len(clean_Name), len(clean_Score), len(clean_Category), len(clean_Price_Range))
        clean_Name = clean_Name[:min_length]
        clean_Score = clean_Score[:min_length]
        clean_Category = clean_Category[:min_length]
        clean_Price_Range = clean_Price_Range[:min_length]
        
        # Store the data in a DataFrame
        new_rows_df = pd.DataFrame({
            'Name': clean_Name,
            'Score': clean_Score,
            'Category': clean_Category,
            'Price_Range': clean_Price_Range,
        })

        df = pd.concat([df, new_rows_df], ignore_index=True)
        
        

        url = f'https://foursquare.com/explore?mode=url&{n}&offset={i}'
    
driver.quit()



In [13]:
df

,Name,Score,Category,Price_Range
0,1. شاهي ابو وليد,9.5,Café,$
1,2. Aseeb Najdi Restaurant,9.5,Restaurant,$
2,3. دوّار السّعادة,9.4,Breakfast,$
3,4. McDonald's,9.4,Fast Food,$$
4,5. SHiRO,9.3,Japanese,$$$
...,...,...,...,...
2295,24. Armin,9.2,Eastern European,$$
2296,25. AlKadesiya Bakery,9.2,Bakery,$
2297,26. Armin,9.2,Mediterranean,$
2298,27. BLANCA,9.2,Pizza,$


In [14]:
df.to_csv('data.csv')